# Imports

In [ ]:
import os
import sys

sys.path.append("..")
from pathlib import Path
from typing import Optional, Dict, List, Union

import os
import torch
from torch.utils.data import random_split
import numpy as np
import pandas as pd

from utils.train_utils import get_model
from utils.coba_dataset import COBA

from models.test import test_img

# Steps

1. Get paths of models (given a directory)
1. Store paths of models
1. For each model...
   1. Load into memory
   2. Test model
   3. Compare to the current best models for each performance metric, replacing if necessary
1. Save best models to `best_models.csv`

# Dynamically Selecting the Best Models

## 1. Get paths of models (given a directory)

In [ ]:
## Simulate the args like in the `main_*.py` files
class ARGS:
    # federated arguments
    # epochs:int = 1000         # rounds of training
    epochs: int = 10  # rounds of training
    train_test_same: int = 0  # use same testing for
    num_users: int = 100  # number of users: K
    shard_per_user: int = 2  # classes per user
    frac: float = 0.1  # the fraction of clients: C
    local_ep: int = 1  # the number of local epochs: E
    local_bs: int = 10  # local batch size: B
    bs: int = 128  # test batch size
    lr: float = 0.01  # learning rate
    # results_save:str = "run1"
    momentum: float = 0.5  # SGD momentum (default: 0.5)
    # gpu:int = 0
    split: str = "user"  # train-test split type, user or sample
    # grad_norm:str           # use_gradnorm_avging
    local_ep_pretrain: int = 0  # the number of pretrain local ep
    lr_decay: float = 1.0  # learning rate decay per round

    # model arguments
    model: str = "cnn"  # model name
    kernel_num: int = 9  # number of each kind of kernel
    kernel_sizes: str = "3,4,5"  # comma-separated kernel size to use for convolution
    norm: str = "batch_norm"  # batch_norm, layer_norm, or None
    num_filters: int = 32  # number of filters for conv nets
    max_pool: str = True  # whether use max pooling rather than strided convolutions
    num_layers_keep: int = 1  # number layers to keep

    # other arguments
    dataset: str = "coba"  # name of dataset
    log_level: str = "warning"  # level of logger
    iid: bool = True  # "store_true" #whether iid or not
    num_classes: int = 14  # number of classes
    num_channels: int = 3  # number of channels of images RGB
    gpu: int = 0  # GPU ID, -1 for CPU
    stopping_rounds: int = 10  # rounds of early stopping
    verbose: bool = True  # "store_true"
    print_freq: int = 100  # print loss frequency during training
    seed: int = 1  # random seed (default:1)
    test_freq: int = 1  # how often to test on val set
    load_fed: str = ""  # define pretrained federated model path
    results_save: str = "run1"  # define fed results save folder
    start_saving: int = 0  # when to start saving models


args = ARGS()

args.device = torch.device(
    "cuda:{}".format(args.gpu)
    if torch.cuda.is_available() and args.gpu != -1
    else "cpu"
)

args.num_users, args.device

In [ ]:
from utils.train_utils import dynamic_model_selector_and_saver

In [ ]:
args.num_users = 98
args.frac = "0.3"
args.iid = False
args.log_level = "info"  # "debug" might crash the notebook
args.seed = 0
args.results_save = "coba_fedavg_bestcase_run2"

base_dir: Path = Path(
    Path.cwd().parent,
    "save",
    "coba_legacy",
    f"{args.model}_iid{args.iid}_num{args.num_users}_C{args.frac}_le{args.local_ep}",
    f"shard{args.shard_per_user}",
)

base_dir = Path(base_dir, f"seed{args.seed}_{args.results_save}")

dynamic_model_selector_and_saver(args=args, base_dir=base_dir)

In [ ]:
# SCENARIO: str = "best"  # the scenario we're interested in
# SEED: int = 0  # the seed of the experiment we're interested in analyzing
# chosen_scenario_dir: Optional[Path] = None
# experiment_run_dir: Path = Path(
#     Path.cwd().parent, "save", "coba_legacy"
# )  # could also be: "coba", "mnist", "cifar10"

In [ ]:
# # Get models according to the chosen scenario
# async_fl_scenarios: Dict[str, str] = {"0.3": "best", "0.5": "average", "1.0": "worst"}

# print("Async FL Scenarios:")
# for dir in experiment_run_dir.glob("*"):
#     scenario_percent: str = (
#         dir.as_posix().split(os.sep)[-1].split("_")[-2].replace("C", "")
#     )
#     print(f"\t{scenario_percent} -> {async_fl_scenarios[scenario_percent]}")

#     if async_fl_scenarios[scenario_percent] == SCENARIO:
#         chosen_scenario_dir = dir

# print(f"{SCENARIO.title()} case scenario directory: '{chosen_scenario_dir.as_posix()}'")

In [ ]:
# # Get paths to models in the chosen scenario
# all_chosen_scenario_runs_dir: Path = Path(chosen_scenario_dir, "shard2")
# all_models_dir: Optional[Path] = None

# for dir in all_chosen_scenario_runs_dir.glob("*"):
#     seed: int = int(dir.as_posix().split(os.sep)[-1].split("_")[0].split("d")[-1])

#     if SEED == seed:
#         all_models_dir = Path(dir, "fed")

# # Raise error if no directory is found with the given SEED
# if all_models_dir is None:
#     raise FileNotFoundError(
#         f"Directory with the provided seed '{SEED}' does not exist. Please choose a different one and try again."
#     )


# print(
#     [
#         model_file.as_posix().split(os.sep)[-1]
#         for model_file in all_models_dir.glob("*.pt")
#     ]
# )

## 2. Store paths of models

In [ ]:
# # model_paths:List[Path] = sorted([model_file for model_file in all_models_dir.glob("*.pt")],key=lambda s: int(s.as_posix().split(os.sep)[-1].split("_")[-1].replace(".pt","")))
# model_paths: List[Path] = [model_file for model_file in all_models_dir.glob("*.pt")]

# REMOVE_BEST: bool = True  # This is to remove the presumed "best" model files
# if REMOVE_BEST:
#     model_paths = np.array(model_paths)[
#         list("best_" not in model_path.as_posix() for model_path in model_paths)
#     ].tolist()

# model_paths

## 3. For each model...

1. Load into memory
2. Test model
3. Compare to the current best models for each performance metric, replacing if necessary

In [ ]:
# class ChosenModel:
#     def __init__(
#         self, path: Path, performance_metrics: Dict[str, float], main_metric: str
#     ) -> None:
#         self.path: Path = path
#         self.performance_metrics: Dict[str, float] = performance_metrics
#         self.main_metric: str = main_metric
#         self.main_metric_value: float = performance_metrics[main_metric]


# def update_metric_value(old_val: float, new_val: float, metric: str) -> bool:
#     return new_val < old_val if metric == "loss" else new_val > old_val

In [ ]:
# metrics: List[str] = ["accuracy", "loss", "f1", "precision", "recall"]
# best_models: Dict[str, Optional[ChosenModel]] = {metric: None for metric in metrics}

# coba_dataset: COBA = COBA(root="../data/coba", download=True)

# ## Create training and testing data -- method 1
# train_size = int(0.8 * len(coba_dataset))
# test_size = len(coba_dataset) - train_size
# _, test_dataset = random_split(dataset=coba_dataset, lengths=[train_size, test_size])

# test_dataset = test_dataset.dataset

In [ ]:
# for model_path in model_paths:
#     # Load into memory
#     model = get_model(args)

#     model.load_state_dict(
#         torch.load(model_path)
#     ) if args.device.type != "cpu" else model.load_state_dict(
#         torch.load(model_path, map_location=torch.device("cpu"))
#     )

#     # Test model
#     acc_test, loss_test, f1_test, precision_test, recall_test = test_img(
#         model, test_dataset, args
#     )
#     results: Dict[str, float] = {
#         metric: value
#         for metric, value in zip(
#             metrics, [acc_test, loss_test, f1_test, precision_test, recall_test]
#         )
#     }

#     # Compare to the current best models for each performance metric, replacing if necessary
#     for metric, metric_value in results.items():
#         if best_models[metric] is None or update_metric_value(
#             old_val=best_models[metric].main_metric_value,
#             new_val=metric_value,
#             metric=metric,
#         ):
#             best_models[metric]: ChosenModel = ChosenModel(
#                 path=model_path, performance_metrics=results, main_metric=metric
#             )

In [ ]:
# for key, model in best_models.items():
#     print(f"Key: {key.capitalize()}")
#     print(f"\tBest Model Filename: {model.path.as_posix().split(os.sep)[-1]}")
#     print(f"\tBest Model Main Metric Value: {model.main_metric_value}")
#     print(f"\tBest Model All Metric Values:")
#     for metric, value in model.performance_metrics.items():
#         print(f"\t\t{metric.capitalize():9}:\t{value}")

## 4. Save best models

In [ ]:
# def reformat_model_for_dataframe(model: ChosenModel) -> Dict[str, Union[str, float]]:
#     model_dict: Dict[str, Union[str, float]] = {
#         "name": model.path.as_posix().split(os.sep)[-1],
#         "main_metric": model.main_metric,
#     }
#     model_dict.update(
#         **{metric: value for metric, value in model.performance_metrics.items()},
#         path=model.path.as_posix(),
#     )
#     return model_dict

In [ ]:
# best_models_filename: str = "best_models.csv"

# models_dataframe_list: List[Dict[str, Union[str, float]]] = [
#     reformat_model_for_dataframe(model=model) for model in best_models.values()
# ]
# models_dataframe_list[0]

In [ ]:
# best_models_df = pd.DataFrame(models_dataframe_list)
# best_models_df

In [ ]:
# best_models_df.to_csv(path_or_buf=Path(best_models_filename), index=False, header=True)